# Importing hugging face transformers and libraries



In [12]:
!pip install transformers datasets pandas scikit-learn

# Import necessary libraries
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split  # Import train_test_split
from torch.utils.data import DataLoader


# Setting the device

In [13]:

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")


Using device: cuda


# Loading tokenizer and model from hf

In [14]:
# 1. Load Tokenizer and Model for BERT
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(DEVICE)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# Checkpoint Dir

In [15]:
# Create a directory for saving checkpoints
CHECKPOINT_DIR = '/content/checkpoints'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)


# Load clean training data

In [1]:
!pip install datasets
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset from Hugging Face
splits = {'train': 'train.jsonl', 'test': 'test.jsonl'}
df = pd.read_json("hf://datasets/SetFit/hate_speech_offensive/" + splits["train"], lines=True)

# Map the labels: 0 for "neither" (nothate) and 1 for anything offensive (hate)
df['label'] = df['label_text'].apply(lambda x: 0 if x == 'neither' else 1)

# Keep only the 'text' and 'label' columns to match the previous format
df = df[['text', 'label']]

# Remove mentions, hashtags, links, and "RT" from the text
df['text'] = df['text'].str.replace(r'@\w+', '', regex=True)   # Remove mentions
df['text'] = df['text'].str.replace(r'#\w+', '', regex=True)   # Remove hashtags
df['text'] = df['text'].str.replace(r'http\S+|www\S+|t.co/\S+', '', regex=True)  # Remove links
df['text'] = df['text'].str.replace(r'\bRT\b', '', regex=True)
df['text'] = df['text'].str.replace(':', '', regex=True)  # Remove "RT" for retweets

# Drop rows with any missing values (if any)
df.dropna(inplace=True)

# Split into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Display the cleaned dataset format
print(train_df.head(15))


# Convert the DataFrames to lists for tokenization
train_texts = train_df['text'].tolist()
train_labels = train_df['label'].tolist()
val_texts = val_df['text'].tolist()
val_labels = val_df['label'].tolist()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
                                                    text  label
6179         Got a badass bitch just bouncin on my dick       1
19957  ,,,,,, |  | Blonde tranny who

# Tokenization

In [ ]:
# Tokenize the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)


# Data Prep

In [ ]:
class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = HateSpeechDataset(train_encodings, train_labels)
val_dataset = HateSpeechDataset(val_encodings, val_labels)



# Data Loaders

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Training Setup

In [ ]:
from transformers import AdamW

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Training Loop

In [ ]:
from transformers import AdamW
from torch.cuda.amp import GradScaler, autocast
optimizer = AdamW(model.parameters(), lr=5e-5)
scaler = GradScaler()

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        # Move input tensors to the GPU
        for key in batch:
            batch[key] = batch[key].to(DEVICE)

        optimizer.zero_grad()

        # Automatic Mixed Precision
        with autocast():
            outputs = model(**batch)
            loss = outputs.loss

        # Scale the loss for mixed precision
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Free up memory
        torch.cuda.empty_cache()

    print(f"Epoch {epoch + 1} finished.")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-29-c6f40cf45cac>:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-29-c6f40cf45cac>:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1 finished.
Epoch 2 finished.
Epoch 3 finished.


# Saving the model

In [ ]:
# After your training loop
output_dir = "/drive/MyDrive/training_v1"  # Specify your save path

# Save the model
model.save_pretrained(output_dir)

# Save the tokenizer
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")


Model and tokenizer saved to /drive/MyDrive/training_v1
